In [1]:
from pyagamo.players import ClonalSelection
from pyagamo import AGAMO
import time
import matplotlib.pyplot as plt
import Pyro4

In [2]:
max_eval = 10000
npop = 25
change_iter = 1
next_iter = -1
max_front = 100
player_parm = { "nclone": 15, "mutate_args": [0.45, 0.9, 0.01], 'sup': 0.0}

In [3]:
obj1 = Pyro4.Proxy('PYRO:obj_6a8948e30ab5418182fb4061d526b8df@localhost:49228')
obj2 = Pyro4.Proxy('PYRO:obj_f46c485dff6f468b939b29d859d218bc@localhost:49228')
obj1_m = Pyro4.Proxy('PYRO:obj_491a5899518e49d8857a3122706caf32@localhost:49228')
obj2_m = Pyro4.Proxy('PYRO:obj_45350d6084bd404f8c5ed7c15495c4ed@localhost:49228')
objs = [obj1, obj2]
objs_m = [obj1_m, obj2_m]

In [4]:
players = [ClonalSelection(i, npop, player_parm, transport='tcp') for i in range(2)]

In [5]:
agamo = AGAMO(max_eval, change_iter, next_iter, max_front, transport='tcp')
agamo.add_objectives(objs, objs_m)
agamo.add_players(players)
agamo.init()

In [6]:
agamo.start_optimize()
res=agamo.get_results()
agamo.close()

In [7]:
res

In [8]:
ax = plt.figure('Front Pareto', figsize=(6, 6)).add_subplot(111)
front_eval = res['front_eval']
ax.scatter(front_eval[:, 0], front_eval[:, 1], marker='o', label=str(max_eval))
ax.grid(True)
plt.legend()
plt.show()

In [9]:
agamo = AGAMO(max_eval, change_iter, next_iter, max_front)
agamo.add_objectives(objs, objs_m)
agamo.add_players(players)
agamo.init()

In [10]:
agamo.start_optimize(thread=True)
res=agamo.get_results()
while not res.get('stop_flag', False):
    res=agamo.get_results()
    front_eval = res.get('front_eval', None)
    if (front_eval is not None) and len(front_eval)>0:
        ax = plt.figure('Front Pareto', figsize=(6, 6)).add_subplot(111)
        ax.grid(True)
        ax.scatter(front_eval[:, 0], front_eval[:, 1], marker='o', label=res['evaluations']+res['evaluations_m'])
        plt.legend()
        plt.show()
    time.sleep(1)
    #print(res.get('iterations', 'brak'))
agamo.close()

In [11]:
res